In [1]:
import pandas as pd
import os
import numpy as np
folder_path = "Data/latest_inputs"  # Make sure the folder path is correct

# Read each CSV file into a DataFrame
df_price = pd.read_csv(os.path.join(folder_path, "Prices.csv"))
df_chargesvar = pd.read_csv(os.path.join(folder_path, "Charges_var.csv"))
df_prod = pd.read_csv(os.path.join(folder_path, "Production.csv"))
df_plantation = pd.read_csv(os.path.join(folder_path, "plantation.csv"))
df_month_index = pd.read_csv(os.path.join(folder_path, "month_index.csv"))
df_sim=pd.read_csv(os.path.join(folder_path, "Simulation.csv"))
# Now you can work with these DataFrames as needed


In [16]:
df_prod

,Scénario,Culture,Couple,variété 23-24,Type de plantations 23-24,Mois,Délai pour début de production,Durée de production en semaine,W1,W2,...,W28,W29,W30,W31,W32,W33,W34,W35,W36,W37
0,1,Framboise,YAZMIN-TP,YAZMIN,TP,Juin,16,16,63.945946,525.567568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Framboise,YAZMIN-TP,YAZMIN,TP,Juillet,16,16,545.205405,324.194595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Framboise,YAZMIN-TP,YAZMIN,TP,Aout,15,16,57.551351,473.010811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Framboise,YAZMIN-MD+CB,YAZMIN,MD+CB,Mai,17,11,9.243410,387.196166,...,414.70625,417.8125,615.125,971.425,1982.1375,2298.7125,2226.6125,2009.7,1298.9375,228.375
4,5,Framboise,YAZMIN-MD+CB,YAZMIN,MD+CB,Avril,17,11,9.243410,387.196166,...,414.70625,417.8125,615.125,971.425,1982.1375,2298.7125,2226.6125,2009.7,1298.9375,228.375
5,8,Framboise,Adelita-TP,Adelita,TP,Juin,18,12,26.250000,164.937500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,9,Framboise,Adelita-TP,Adelita,TP,Juillet,20,12,26.250000,164.937500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10,Framboise,Adelita-TP,Adelita,TP,Aout,21,11,26.250000,164.937500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,11,Mure,VICTORIA-Long canne ST,VICTORIA,Long canne ST,Novembre,13,11,212.867241,539.563793,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,12,Mure,LAURITA-Long canne ST,LAURITA,Long canne ST,Octobre,12,11,180.395967,457.257452,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
scenario_delai_dict = {}
scenario_duree_dict = {}
scenario_mois_dict = {}
scenario_culture={}

for index, row in df_prod.iterrows():
    scenario = row['Scénario']
    delai = row['Délai pour début de production']
    duree = row['Durée de production en semaine']
    mois = row['Mois']
    culture=row['Culture']
    scenario_delai_dict[scenario] = delai
    scenario_duree_dict[scenario] = duree
    scenario_mois_dict[scenario] = mois
    scenario_culture[scenario] =culture

In [15]:
price={}
price["Framboise"]=np.array(df_price.iloc[0,1:])
price["Mure"]=np.array(df_price.iloc[1,1:])

In [4]:
secteur_serre_dict = {}

# Iterate over the rows of the DataFrame
for index, row in df_sim.iterrows():
    secteur = row['secteur']
    serre = row['serre']
    
    # If the secteur is not in the dictionary yet, create a new list
    # Otherwise, append the serre to the existing list
    if secteur not in secteur_serre_dict:
        secteur_serre_dict[int(secteur)] = [int(serre)]
    else:
        secteur_serre_dict[int(secteur)].append(int(serre))
num_serre=df_sim.shape[0]
num_sect=len(secteur_serre_dict)
num_scenario=19

In [7]:
import gurobipy as gp
from gurobipy import GRB

try:
    # Create a new model
    m = gp.Model("portfolio")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")
    
    #Choice of Scenario par serre variables
    variables={}
    for i in range(num_serre):
        for j in range(num_scenario):
            variables[(i, j)] = m.addVar(vtype=GRB.BINARY, name=f'x_{i}_{j}')
    #These are the constraints for the variables I defined Above
    #Meme scenario par secteur
    for i in range(num_sect):
        ref=secteur_serre_dict[i][0]
        for j in secteur_serre_dict[i]:
            for k in range(num_scenario):
                m.addConstr(variables[(j,k)] ==variables[(ref,k)], f'c_0_{j}_{i}')
    #Cette contrainte est obtenue par la contraposé de la cinquième contrainte
    for i in range(num_sect):
        if i!=6:
            for j in secteur_serre_dict[i]:
                m.addConstr(variables[(j,5)]==0,f'c_1_{j}_{i}')
    #For now, we have done constraints 2 and 5, now let's do the rest
    
    #Variables de semaines
    semaines={}
    for i in range(num_serre):
        semaines[i]=m.addVar(vtype=GRB.CONTINUOUS, name=f's_{i}')
    choix_semaines={}
    for i in range(num_serre):
        for t in range(1,91):
            choix_semaines[(i,t)]=m.addVar(vtype=GRB.BINARY, name=f'cs_{i}_{t}')
        
    prod={}
    for i in range(num_serre):
        for j in range(num_scenario):
            for t in range(1,91):
                prod[(i,j,t)]=m.addVar(vtype=GRB.CONTINUOUS, name=f'p_{i}_{j}_{t}')
    #This constraint still needs to be written
    for i in range(num_serre):
        for j in range(num_scenario):
            for t in range(1,91):
                m.addConstr(prod[(i,j,t)]==np.dot(price[scenario_culture[j]][t+scenario_delai_dict[j],t+scenario_delai_dict[j]
                                                                             +t+scenario_duree_dict[j]]
                                                  ,))


                
    for i in range(num_serre):
        for j in range(num_scenario):
            m.addConstr(variables[(j,5)]==0,f'c_1_{j}_{i}')
        
    
    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"Obj: {m.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7300U CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.04s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.12 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x 1
y 0
z 1
Obj: 3
